# 🎯 Clustering con K-Means
## Segmentación de Clientes con Tarjetas de Crédito

---

### 📌 Información del Notebook

- **Objetivo**: Aplicar clustering K-Means para segmentar clientes
- **Método**: K-Means con validación por Silhouette Score
- **Autor**: [Tu Nombre]
- **Fecha**: Enero 2026

---

### 🎯 Contenido

1. Carga de datos procesados
2. Determinación del número óptimo de clusters (Método del Codo)
3. Validación con Silhouette Score
4. Aplicación de K-Means
5. Visualización de clusters
6. Caracterización de segmentos
7. Interpretación de negocio
8. Exportación de resultados


### Importar librerías necesarias


In [ ]:
# Importamos todas las librerías necesarias para clustering, visualización y análisis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Librerías importadas correctamente")


---
## 📁 1. Carga de Datos Procesados

### Cargar datos normalizados y con PCA


In [ ]:
# Cargamos los datos procesados del notebook anterior
# Necesitamos ambos: normalizados (para clustering) y PCA (para visualización)
df_normalizado = pd.read_csv('../datos/datos_normalizados.csv')
df_pca = pd.read_csv('../datos/datos_pca.csv')
df_original = pd.read_csv('../datos/datos_procesados.csv')

print("="*70)
print("📊 DATOS CARGADOS")
print("="*70)
print(f"Datos normalizados: {df_normalizado.shape}")
print(f"Datos PCA: {df_pca.shape}")
print(f"Datos originales: {df_original.shape}")
print("="*70)

---
## 📊 2. Método del Codo (Elbow Method)

### Determinar el número óptimo de clusters


In [ ]:
# El Método del Codo nos ayuda a encontrar el K óptimo
# Calculamos la inercia (suma de distancias al cuadrado) para diferentes K
# El "codo" en la curva indica el número óptimo de clusters

# Probamos de 2 a 10 clusters
rango_k = range(2, 11)
inercias = []

print("🔄 Calculando inercias para diferentes valores de K...")
print("="*70)

for k in rango_k:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=300)
    kmeans.fit(df_normalizado)
    inercias.append(kmeans.inertia_)
    print(f"K={k:2d} → Inercia: {kmeans.inertia_:>10,.2f}")

print("="*70)
print("✅ Cálculo completado")

### Visualizar el Método del Codo


In [ ]:
# Graficamos la curva de inercia vs número de clusters
# El "codo" (punto donde la pendiente cambia drásticamente) indica el K óptimo
plt.figure(figsize=(12, 6))
plt.plot(rango_k, inercias, marker='o', linewidth=2, markersize=10, color='steelblue')

# Marcamos cada punto con su valor
for i, (k, inercia) in enumerate(zip(rango_k, inercias)):
    plt.text(k, inercia, f'{inercia:,.0f}', ha='center', va='bottom', fontsize=9)

plt.title('Método del Codo - Determinación del K Óptimo', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Número de Clusters (K)', fontsize=12)
plt.ylabel('Inercia (Suma de Distancias al Cuadrado)', fontsize=12)
plt.xticks(rango_k)
plt.grid(True, alpha=0.3)

# Añadimos líneas de referencia
plt.axvline(x=4, color='red', linestyle='--', alpha=0.5, label='K=4 (posible codo)')
plt.legend()

plt.tight_layout()
plt.savefig('../resultados/graficos/metodo_codo.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico guardado en: resultados/graficos/metodo_codo.png")

---
## 📈 3. Validación con Silhouette Score

### Calcular Silhouette Score para cada K


In [ ]:
# El Silhouette Score mide qué tan bien están separados los clusters
# Rango: [-1, 1]
# - Cercano a 1: clusters bien separados y compactos
# - Cercano a 0: clusters solapados
# - Negativo: puntos mal asignados

silhouette_scores = []

print("🔄 Calculando Silhouette Scores...")
print("="*70)

for k in rango_k:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=300)
    labels = kmeans.fit_predict(df_normalizado)
    score = silhouette_score(df_normalizado, labels)
    silhouette_scores.append(score)
    print(f"K={k:2d} → Silhouette Score: {score:.4f}")

print("="*70)

# Identificamos el K con mejor score
mejor_k = rango_k[np.argmax(silhouette_scores)]
mejor_score = max(silhouette_scores)
print(f"\n✅ Mejor K según Silhouette Score: {mejor_k} (Score: {mejor_score:.4f})")

### Visualizar Silhouette Scores


In [ ]:
# Graficamos los Silhouette Scores para cada K
# Esto nos ayuda a confirmar el K óptimo
plt.figure(figsize=(12, 6))
plt.plot(rango_k, silhouette_scores, marker='s', linewidth=2, markersize=10, color='green')

# Marcamos cada punto
for i, (k, score) in enumerate(zip(rango_k, silhouette_scores)):
    plt.text(k, score, f'{score:.3f}', ha='center', va='bottom', fontsize=9)

plt.title('Silhouette Score por Número de Clusters', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Número de Clusters (K)', fontsize=12)
plt.ylabel('Silhouette Score', fontsize=12)
plt.xticks(rango_k)
plt.grid(True, alpha=0.3)

# Marcamos el mejor K
plt.axvline(x=mejor_k, color='red', linestyle='--', alpha=0.5, label=f'Mejor K={mejor_k}')
plt.legend()

plt.tight_layout()
plt.savefig('../resultados/graficos/silhouette_score.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico guardado en: resultados/graficos/silhouette_score.png")

---
## 🎯 4. Aplicar K-Means con K Óptimo

### Entrenar modelo K-Means final


In [ ]:
# Aplicamos K-Means con el número óptimo de clusters
# Usamos el K que mejor balance tiene entre método del codo y silhouette score
# Típicamente será K=3 o K=4

# Puedes ajustar este valor según tus resultados
k_optimo = 4  # Ajusta según tus gráficos anteriores

print(f"🎯 Aplicando K-Means con K={k_optimo}")
print("="*70)

# Entrenamos el modelo final
kmeans_final = KMeans(
    n_clusters=k_optimo,
    random_state=42,
    n_init=10,
    max_iter=300
)

# Predecimos clusters
clusters = kmeans_final.fit_predict(df_normalizado)

print(f"✅ Modelo entrenado exitosamente")
print(f"   Inercia final: {kmeans_final.inertia_:,.2f}")
print(f"   Iteraciones: {kmeans_final.n_iter_}")
print("="*70)

### Calcular métricas de calidad del clustering


In [ ]:
# Calculamos múltiples métricas para evaluar la calidad del clustering
# - Silhouette Score: separación entre clusters
# - Davies-Bouldin Index: compacidad y separación (menor es mejor)
# - Calinski-Harabasz Score: ratio entre dispersión inter e intra-cluster (mayor es mejor)

silhouette = silhouette_score(df_normalizado, clusters)
davies_bouldin = davies_bouldin_score(df_normalizado, clusters)
calinski = calinski_harabasz_score(df_normalizado, clusters)

print("\n📊 MÉTRICAS DE CALIDAD DEL CLUSTERING")
print("="*70)
print(f"Silhouette Score:        {silhouette:.4f}  (Rango: [-1, 1], mejor cercano a 1)")
print(f"Davies-Bouldin Index:    {davies_bouldin:.4f}  (Menor es mejor)")
print(f"Calinski-Harabasz Score: {calinski:.2f}  (Mayor es mejor)")
print("="*70)

# Interpretación
if silhouette > 0.5:
    print("✅ Excelente separación entre clusters")
elif silhouette > 0.3:
    print("✅ Buena separación entre clusters")
else:
    print("⚠️ Separación moderada, considerar ajustar K")

### Añadir clusters a los dataframes


In [ ]:
# Añadimos la columna de cluster a todos nuestros dataframes
# Esto nos permite analizar los segmentos desde diferentes perspectivas
df_normalizado['Cluster'] = clusters
df_pca['Cluster'] = clusters
df_original['Cluster'] = clusters

print("✅ Columna 'Cluster' añadida a todos los dataframes")

# Contamos cuántos clientes hay en cada cluster
print("\n📊 DISTRIBUCIÓN DE CLIENTES POR CLUSTER")
print("="*70)
for i in range(k_optimo):
    n_clientes = (clusters == i).sum()
    porcentaje = (n_clientes / len(clusters)) * 100
    print(f"Cluster {i}: {n_clientes:>5} clientes ({porcentaje:>5.2f}%)")
print("="*70)

---
## 📊 5. Visualización de Clusters

### Visualización 2D con PCA (PC1 vs PC2)


In [ ]:
# Visualizamos los clusters en 2D usando los primeros dos componentes principales
# Esto nos permite ver la separación espacial de los grupos
plt.figure(figsize=(14, 8))

# Definimos colores para cada cluster
colores = ['red', 'blue', 'green', 'orange', 'purple', 'brown']
markers = ['o', 's', '^', 'D', 'v', 'p']

# Graficamos cada cluster
for i in range(k_optimo):
    cluster_data = df_pca[df_pca['Cluster'] == i]
    plt.scatter(
        cluster_data['PC1'],
        cluster_data['PC2'],
        c=colores[i],
        marker=markers[i],
        s=50,
        alpha=0.6,
        edgecolors='black',
        linewidth=0.5,
        label=f'Cluster {i} (n={len(cluster_data)})'
    )

# Marcamos los centroides
centroides_pca = []
for i in range(k_optimo):
    centroide = df_pca[df_pca['Cluster'] == i][['PC1', 'PC2']].mean()
    centroides_pca.append(centroide)
    plt.scatter(
        centroide['PC1'],
        centroide['PC2'],
        c=colores[i],
        marker='X',
        s=300,
        edgecolors='black',
        linewidth=2,
        label=f'Centroide {i}'
    )

plt.title(f'Visualización de {k_optimo} Clusters - PCA 2D', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Componente Principal 1 (PC1)', fontsize=12)
plt.ylabel('Componente Principal 2 (PC2)', fontsize=12)
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../resultados/graficos/pca_2d_clusters.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico guardado en: resultados/graficos/pca_2d_clusters.png")

### Visualización 3D con PCA (PC1, PC2, PC3)


In [ ]:
# Visualizamos en 3D si tenemos al menos 3 componentes principales
# Esto da una perspectiva más completa de la separación de clusters
if df_pca.shape[1] >= 4:  # 3 PCs + Cluster
    fig = plt.figure(figsize=(14, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Graficamos cada cluster
    for i in range(k_optimo):
        cluster_data = df_pca[df_pca['Cluster'] == i]
        ax.scatter(
            cluster_data['PC1'],
            cluster_data['PC2'],
            cluster_data['PC3'],
            c=colores[i],
            marker=markers[i],
            s=50,
            alpha=0.6,
            edgecolors='black',
            linewidth=0.5,
            label=f'Cluster {i}'
        )
    
    # Centroides en 3D
    for i in range(k_optimo):
        centroide = df_pca[df_pca['Cluster'] == i][['PC1', 'PC2', 'PC3']].mean()
        ax.scatter(
            centroide['PC1'],
            centroide['PC2'],
            centroide['PC3'],
            c=colores[i],
            marker='X',
            s=300,
            edgecolors='black',
            linewidth=2
        )
    
    ax.set_title(f'Visualización 3D de {k_optimo} Clusters', fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('PC1', fontsize=11)
    ax.set_ylabel('PC2', fontsize=11)
    ax.set_zlabel('PC3', fontsize=11)
    ax.legend(loc='best', fontsize=9)
    
    plt.tight_layout()
    plt.savefig('../resultados/graficos/pca_3d_clusters.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Gráfico 3D guardado en: resultados/graficos/pca_3d_clusters.png")
else:
    print("⚠️ No hay suficientes componentes para visualización 3D")

### Visualización interactiva con Plotly


In [ ]:
# Creamos una visualización interactiva con Plotly
# Esto permite hacer zoom, rotar y explorar los clusters de manera dinámica
fig = px.scatter(
    df_pca,
    x='PC1',
    y='PC2',
    color='Cluster',
    title=f'Clusters Interactivos - K={k_optimo}',
    color_continuous_scale='Viridis',
    hover_data=['PC1', 'PC2']
)

fig.update_traces(marker=dict(size=8, opacity=0.7, line=dict(width=1, color='white')))
fig.update_layout(
    width=900,
    height=600,
    font=dict(size=12),
    title_font_size=16
)

fig.show()

print("✅ Gráfico interactivo generado (explora con el mouse)")

### Distribución de clientes por cluster (Gráfico de Pastel)


In [ ]:
# Creamos un gráfico de pastel para ver la proporción de cada cluster
# Esto ayuda a identificar si hay clusters dominantes o balanceados
cluster_counts = df_original['Cluster'].value_counts().sort_index()

plt.figure(figsize=(10, 8))
plt.pie(
    cluster_counts,
    labels=[f'Cluster {i}\n({cluster_counts[i]} clientes)' for i in cluster_counts.index],
    autopct='%1.1f%%',
    colors=colores[:k_optimo],
    startangle=90,
    explode=[0.05] * k_optimo,
    shadow=True
)
plt.title(f'Distribución de Clientes por Cluster (K={k_optimo})', 
          fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('../resultados/graficos/distribucion_clusters.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico guardado en: resultados/graficos/distribucion_clusters.png")

---
## 🔍 6. Caracterización de Clusters

### Estadísticas por cluster


In [ ]:
# Calculamos estadísticas descriptivas para cada cluster
# Esto nos ayuda a entender las características de cada segmento
print("📊 ESTADÍSTICAS POR CLUSTER")
print("="*100)

estadisticas_clusters = df_original.groupby('Cluster').agg(['mean', 'median', 'std', 'min', 'max'])

# Mostramos las medias de cada variable por cluster
print("\n📈 MEDIAS POR CLUSTER:\n")
medias = df_original.groupby('Cluster').mean()
print(medias.round(2))

In [ ]:
# Guardamos las estadísticas completas en CSV
estadisticas_clusters.to_csv('../resultados/metricas/estadisticas_clusters.csv')
print("\n✅ Estadísticas guardadas en: resultados/metricas/estadisticas_clusters.csv")

### Heatmap de características por cluster


In [ ]:
# Creamos un heatmap para visualizar las diferencias entre clusters
# Normalizamos por columna para ver patrones relativos
medias_norm = (medias - medias.mean()) / medias.std()

plt.figure(figsize=(14, 8))
sns.heatmap(
    medias_norm.T,
    annot=False,
    cmap='RdYlGn',
    center=0,
    linewidths=0.5,
    cbar_kws={'label': 'Valor Normalizado (z-score)'}
)
plt.title('Perfil de Características por Cluster', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Cluster', fontsize=12)
plt.ylabel('Variables', fontsize=12)
plt.tight_layout()
plt.savefig('../resultados/graficos/perfiles_clusters.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico guardado en: resultados/graficos/perfiles_clusters.png")

### Comparación de variables clave entre clusters


In [ ]:
# Comparamos las variables más importantes entre clusters usando boxplots
# Esto muestra la distribución y diferencias entre segmentos
variables_comparar = ['BALANCE', 'PURCHASES', 'CREDIT_LIMIT', 'PAYMENTS', 'CASH_ADVANCE', 'TENURE']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, var in enumerate(variables_comparar):
    # Creamos boxplot por cluster
    data_por_cluster = [df_original[df_original['Cluster'] == i][var] for i in range(k_optimo)]
    
    bp = axes[idx].boxplot(
        data_por_cluster,
        labels=[f'C{i}' for i in range(k_optimo)],
        patch_artist=True
    )
    
    # Coloreamos cada box
    for patch, color in zip(bp['boxes'], colores[:k_optimo]):
        patch.set_facecolor(color)
        patch.set_alpha(0.6)
    
    axes[idx].set_title(f'{var}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Cluster')
    axes[idx].set_ylabel('Valor')
    axes[idx].grid(True, alpha=0.3, axis='y')

plt.suptitle('Comparación de Variables Clave por Cluster', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()


---
## 💡 7. Interpretación de Negocio

### Análisis detallado de cada cluster


In [ ]:
# Analizamos cada cluster en detalle para crear perfiles de negocio
# Identificamos las características distintivas de cada segmento
print("💡 INTERPRETACIÓN DE CLUSTERS")
print("="*100)

for i in range(k_optimo):
    cluster_data = df_original[df_original['Cluster'] == i]
    n_clientes = len(cluster_data)
    porcentaje = (n_clientes / len(df_original)) * 100
    
    print(f"\n{'='*100}")
    print(f"🎯 CLUSTER {i} - {n_clientes} clientes ({porcentaje:.1f}%)")
    print(f"{'='*100}")
    
    # Características principales
    print(f"\n📊 Características Promedio:")
    print(f"  Balance:           ${cluster_data['BALANCE'].mean():>10,.2f}")
    print(f"  Compras:           ${cluster_data['PURCHASES'].mean():>10,.2f}")
    print(f"  Límite Crédito:    ${cluster_data['CREDIT_LIMIT'].mean():>10,.2f}")
    print(f"  Pagos:             ${cluster_data['PAYMENTS'].mean():>10,.2f}")
    print(f"  Adelantos:         ${cluster_data['CASH_ADVANCE'].mean():>10,.2f}")
    print(f"  Antigüedad:        {cluster_data['TENURE'].mean():>10,.1f} meses")
    print(f"  % Pago Completo:   {cluster_data['PRC_FULL_PAYMENT'].mean()*100:>10,.1f}%")
    
    # Comportamiento
    print(f"\n🔍 Comportamiento:")
    usa_adelantos = (cluster_data['CASH_ADVANCE'] > 0).sum() / n_clientes * 100
    compra_frecuente = (cluster_data['PURCHASES_FREQUENCY'] > 0.5).sum() / n_clientes * 100
    paga_completo = (cluster_data['PRC_FULL_PAYMENT'] > 0.5).sum() / n_clientes * 100
    
    print(f"  Usa adelantos:     {usa_adelantos:>6.1f}% de clientes")
    print(f"  Compra frecuente:  {compra_frecuente:>6.1f}% de clientes")
    print(f"  Paga completo:     {paga_completo:>6.1f}% de clientes")

print(f"\n{'='*100}")

### Nomenclatura y perfiles de negocio


In [ ]:
# Asignamos nombres descriptivos a cada cluster basados en sus características
# Estos nombres facilitan la comunicación con stakeholders
print("\n🏷️ NOMENCLATURA DE CLUSTERS")
print("="*100)

# Analiza tus resultados y ajusta estos nombres según las características reales
nombres_clusters = {
    0: "Transactors - Pagadores Completos",
    1: "Revolvers - Alto Balance",
    2: "VIP - Alto Valor",
    3: "Cash Advance Users - Usuarios de Adelantos"
}

descripciones_clusters = {
    0: """
    Clientes que pagan su balance completo regularmente.
    Usan la tarjeta moderadamente y son de bajo riesgo.
    Estrategia: Programas de rewards, cashback.
    """,
    1: """
    Mantienen balance alto y hacen pagos mínimos.
    Generan intereses constantemente.
    Estrategia: Productos de consolidación, educación financiera.
    """,
    2: """
    Alto límite de crédito, compras elevadas, pagos completos.
    Clientes premium de alto valor.
    Estrategia: Servicios exclusivos, upgrades premium.
    """,
    3: """
    Uso frecuente de adelantos en efectivo.
    Posible señal de problemas financieros.
    Estrategia: Alertas tempranas, alternativas de crédito.
    """
}

for i in range(k_optimo):
    print(f"\n🎯 Cluster {i}: {nombres_clusters.get(i, f'Cluster {i}')}")
    print(f"{'─'*100}")
    print(descripciones_clusters.get(i, "Análisis pendiente"))

print("="*100)

# NOTA: Ajusta estos nombres y descripciones según TUS datos reales
print("\n⚠️ IMPORTANTE: Revisa las estadísticas anteriores y ajusta estos nombres")
print("   según las características reales de tus clusters.")

---
## 💾 8. Exportar Resultados

### Guardar datos con clusters asignados


In [ ]:
# Guardamos el dataset final con los clusters asignados
# Esto permite análisis posteriores y uso en producción
df_final = df_original.copy()
df_final['Cluster'] = clusters

# Añadimos nombres de clusters
df_final['Cluster_Nombre'] = df_final['Cluster'].map(nombres_clusters)

# Guardamos en CSV
df_final.to_csv('../datos/datos_con_clusters.csv', index=False)
print("✅ Datos con clusters guardados: datos/datos_con_clusters.csv")

# Guardamos solo los IDs de clientes con sus clusters (si tuviéramos IDs)
# df_resultados = pd.DataFrame({
#     'CUST_ID': df_original.index,  # O tu columna de IDs
#     'Cluster': clusters,
#     'Cluster_Nombre': df_final['Cluster_Nombre']
# })
# df_resultados.to_csv('../resultados/asignacion_clusters.csv', index=False)
# print("✅ Asignación de clusters guardada")

### Guardar métricas de evaluación


In [ ]:
# Guardamos todas las métricas de evaluación en un archivo de texto
# Esto documenta la calidad del modelo para reportes y auditorías
metricas_texto = f"""
{'='*80}
REPORTE DE CLUSTERING - SEGMENTACIÓN DE CLIENTES
{'='*80}

CONFIGURACIÓN:
  - Algoritmo: K-Means
  - Número de clusters: {k_optimo}
  - Random state: 42
  - Datos: {len(df_original)} clientes
  - Variables: {len(df_original.columns)-1}

MÉTRICAS DE CALIDAD:
  - Silhouette Score:        {silhouette:.4f}
  - Davies-Bouldin Index:    {davies_bouldin:.4f}
  - Calinski-Harabasz Score: {calinski:.2f}
  - Inercia:                 {kmeans_final.inertia_:,.2f}

DISTRIBUCIÓN DE CLUSTERS:
"""

for i in range(k_optimo):
    n_clientes = (clusters == i).sum()
    porcentaje = (n_clientes / len(clusters)) * 100
    metricas_texto += f"  Cluster {i}: {n_clientes:>5} clientes ({porcentaje:>5.2f}%)\n"

metricas_texto += f"\n{'='*80}\n"

# Guardar en archivo
with open('../resultados/metricas/metricas_clustering.txt', 'w', encoding='utf-8') as f:
    f.write(metricas_texto)

print("✅ Métricas guardadas en: resultados/metricas/metricas_clustering.txt")
print("\n" + metricas_texto)

---
## 📋 Resumen y Conclusiones

### ✅ Trabajo Completado:

1. **Número óptimo de clusters**: K = 4 (ajustar según tus resultados)
2. **Método de selección**: Método del Codo + Silhouette Score
3. **Calidad del clustering**: Silhouette Score > 0.3 indica buena separación
4. **Segmentos identificados**: 4 grupos distintos de clientes

### 📊 Clusters Identificados:

- **Cluster 0**: Transactors (Pagadores completos)
- **Cluster 1**: Revolvers (Alto balance, pagos mínimos)
- **Cluster 2**: VIP (Alto valor, clientes premium)
- **Cluster 3**: Cash Advance Users (Usuarios de adelantos)

### 💼 Valor de Negocio:

1. **Marketing dirigido**: Campañas específicas por segmento
2. **Gestión de riesgo**: Identificación de clientes de riesgo
3. **Retención**: Estrategias personalizadas por cluster
4. **Productos**: Ofertas adaptadas a cada perfil
5. **Revenue**: Maximización de ingresos por segmento


